https://towardsdatascience.com/a-complete-exploratory-data-analysis-and-visualization-for-text-data-29fb1b96fb6a

In [1]:
import numpy as np
import pandas as pd
import cufflinks
from sklearn.feature_extraction.text import CountVectorizer
from plotly.offline import iplot
cufflinks.go_offline()
cufflinks.set_config_file(world_readable=True, theme='pearl')


In [2]:
df = pd.read_csv("Crowler/booking_reviews.csv")

In [3]:
df[:5]

,title,pos_text,neg_text,ratingValue,bestRating,rate
0,наступної мандрівки до Львова оберемо цей готель.,Відмінне розташування готелю! Дуже близько до ...,"один маленький мінус, але нам це не завдало не...",10.0,10.0,1
1,"Сніданок - великі порціїї, смачно!","Сніданок - великі порціїї, смачно! 1 страва бе...",Сам готель усередині нагадує гуртожиток чи хос...,6.3,10.0,0
2,"Все чудово! За нагоди, обов'язково знову завіт...",Дуже чудовий і затишний отель! Дуже зручне роз...,"Сантехніку (труби , стоки) необхідно перевірит...",9.6,10.0,1
3,"класне розташування, такий собі острівець тиші...","класне розташування, такий собі острівець тиші...",начебто новий ремонт - а велика тріщина над дв...,7.5,10.0,0
4,"Гарне місце для туристів або відрядження, але ...","Ідеальний номер, як для тризіркового готелю - ...",Дуже круті сходи. Складно знайти поруч місце д...,9.2,10.0,1


## Review raiting distribution

In [4]:
df['ratingValue'].iplot(
    kind='hist',
    bins=50,
    xTitle='polarity',
    linecolor='black',
    yTitle='count',
    title='Review rating dictribution (booking.com)')

In [5]:
df['rate'].iplot(
    kind='hist',
    bins=50,
    xTitle='polarity',
    linecolor='black',
    yTitle='count',
    title='Negative, Neutral, Positive')

## Estimate length of text 

In [6]:
df['title_len'] = df['title'].astype(str).apply(len)
df['pos_len'] = df['pos_text'].astype(str).apply(len)
df['neg_len'] = df['neg_text'].astype(str).apply(len)

In [7]:
df['title_len'].iplot(
    kind='hist',
    bins=100,
    xTitle='review length',
    linecolor='black',
    yTitle='count',
    title='Title length distribution')

In [8]:
df['pos_len'].iplot(
    kind='hist',
    bins=100,
    xTitle='review length',
    linecolor='black',
    yTitle='count',
    title='Positive text length distribution')

In [9]:
df['neg_len'].iplot(
    kind='hist',
    bins=100,
    xTitle='review length',
    linecolor='black',
    yTitle='count',
    title='Negative text length distribution')

## The distribution of word count

In [10]:
df['title_word_count'] = df['title'].apply(lambda x: len(str(x).split()))
df['pos_word_count'] = df['pos_text'].apply(lambda x: len(str(x).split()))
df['neg_word_count'] = df['neg_text'].apply(lambda x: len(str(x).split()))

In [11]:
df['title_word_count'].iplot(
    kind='hist',
    bins=100,
    xTitle='word count',
    linecolor='black',
    yTitle='count',
    title='Title Word Count Distribution')

In [12]:
df['pos_word_count'].iplot(
    kind='hist',
    bins=100,
    xTitle='word count',
    linecolor='black',
    yTitle='count',
    title='Positive Review Word Count Distribution')

In [13]:
df['neg_word_count'].iplot(
    kind='hist',
    bins=100,
    xTitle='word count',
    linecolor='black',
    yTitle='count',
    title='Negative Review Text Word Count Distribution')

## The distribution of top unigrams 

In [14]:
df['full_review_text'] = df['title'] + " " + df['pos_text'] + " " + df['neg_text']

### The distribution of top unigrams before removing stop words

In [23]:
def get_top_n_words(corpus, n=None, stop_words=None):
    vec = CountVectorizer(stop_words = stop_words).fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
    return words_freq[:n]

In [24]:
common_words = get_top_n_words(df['full_review_text'], 20)
for word, freq in common_words:
    print(word, freq)
df1 = pd.DataFrame(common_words, columns = ['ReviewText' , 'count'])
df1.groupby('ReviewText').sum()['count'].sort_values(ascending=False).iplot(
    kind='bar', yTitle='Count', linecolor='black', title='Top 20 words in review before removing stop words')


не 921
на 865
дуже 716
та 514
розташування 502
персонал 495
все 444
було 365
що 361
номер 349
але 326
готель 309
номері 293
для 273
до 272
це 263
чисто 251
за 235
сніданок 222
привітний 211


### The distribution of top unigrams after removing stop words

In [25]:
def read_stop_words(file):
    with open(file) as f:
        stop_words = f.read().split('\n')

    return stop_words

In [26]:
uk_stop_words = read_stop_words('ukrainian-stopwords.txt')

In [28]:
common_words = get_top_n_words(df['full_review_text'], 20, uk_stop_words)
for word, freq in common_words:
    print(word, freq)
df1 = pd.DataFrame(common_words, columns = ['ReviewText' , 'count'])
df1.groupby('ReviewText').sum()['count'].sort_values(ascending=False).iplot(
    kind='bar', yTitle='Count', linecolor='black', title='Top 20 words in review before removing stop words')



розташування 502
персонал 495
номер 349
готель 309
номері 293
чисто 251
сніданок 222
привітний 211
зручне 179
готелю 169
місце 163
центрі 141
чистота 124
міста 111
гарний 111
супер 102
ліжко 102
ціна 102
сніданки 101
чудове 96


### The distribution of top bigrams before removing stop words

In [30]:
def get_top_n_bigram(corpus, n=None, stop_words=None):
    vec = CountVectorizer(ngram_range=(2, 2), stop_words=stop_words).fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
    return words_freq[:n]

In [31]:
common_words = get_top_n_bigram(df['full_review_text'], 20)
for word, freq in common_words:
    print(word, freq)
df4 = pd.DataFrame(common_words, columns = ['ReviewText' , 'count'])
df4.groupby('ReviewText').sum()['count'].sort_values(ascending=False).iplot(
    kind='bar', yTitle='Count', linecolor='black', title='Top 20 bigrams in review after removing stop words')

привітний персонал 147
місце розташування 88
не було 79
зручне розташування 73
дуже чисто 54
інтер єр 51
центрі міста 48
не дуже 42
дуже зручне 41
смачний сніданок 41
ванній кімнаті 40
ванна кімната 40
чудове розташування 39
самому центрі 39
до центру 39
все сподобалось 39
те що 39
це не 38
на рецепції 37
приємний персонал 35


### The distribution of top bigrams after removing stop words

In [32]:
common_words = get_top_n_bigram(df['full_review_text'], 20, uk_stop_words)
for word, freq in common_words:
    print(word, freq)
df4 = pd.DataFrame(common_words, columns = ['ReviewText' , 'count'])
df4.groupby('ReviewText').sum()['count'].sort_values(ascending=False).iplot(
    kind='bar', yTitle='Count', linecolor='black', title='Top 20 bigrams in review after removing stop words')

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:301: UserWarning:

Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['дев', 'ласка', 'разу', 'ятий', 'ятнадцятий', 'ятнадцять', 'ять', 'ім'] not in stop_words.



привітний персонал 147
місце розташування 88
зручне розташування 73
інтер єр 51
центрі міста 48
персонал привітний 42
смачний сніданок 41
ванній кімнаті 40
ванна кімната 40
чудове розташування 39
приємний персонал 35
розташування готелю 33
ціна якість 33
ввічливий персонал 32
зручне ліжко 31
площі ринок 27
wi fi 26
чудовий готель 25
обов язково 23
свіжий ремонт 23


### The distribution of Top trigrams before removing stop words

In [33]:
def get_top_n_trigram(corpus, n=None, stop_words=None):
    vec = CountVectorizer(ngram_range=(3, 3), stop_words=stop_words).fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
    return words_freq[:n]


In [35]:
common_words = get_top_n_trigram(df['full_review_text'], 20)
for word, freq in common_words:
    print(word, freq)
df6 = pd.DataFrame(common_words, columns = ['ReviewText' , 'count'])
df6.groupby('ReviewText').sum()['count'].sort_values(ascending=False).iplot(
    kind='bar', yTitle='Count', linecolor='black', title='Top 20 trigrams in review before removing stop words')


на першому поверсі 18
дуже зручне розташування 17
зручне місце розташування 16
до площі ринок 16
дуже привітний персонал 15
близько до центру 15
співвідношення ціна якість 14
дуже смачний сніданок 14
зручне розташування готелю 13
розташування привітний персонал 12
все було чудово 11
номері не було 11
за таку ціну 11
але це не 10
гарне місце розташування 10
на вищому рівні 10
не працював ліфт 10
персонал дуже привітний 9
для тих хто 9
на одну ніч 9


In [36]:
common_words = get_top_n_trigram(df['full_review_text'], 20, uk_stop_words)
for word, freq in common_words:
    print(word, freq)
df6 = pd.DataFrame(common_words, columns = ['ReviewText' , 'count'])
df6.groupby('ReviewText').sum()['count'].sort_values(ascending=False).iplot(
    kind='bar', yTitle='Count', linecolor='black', title='Top 20 trigrams in review after removing stop words')



/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:301: UserWarning:

Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['дев', 'ласка', 'разу', 'ятий', 'ятнадцятий', 'ятнадцять', 'ять', 'ім'] not in stop_words.



зручне місце розташування 16
розташування привітний персонал 15
співвідношення ціна якість 14
зручне розташування готелю 13
співвідношення ціни якості 10
гарне місце розташування 10
розташування центрі міста 9
чисто привітний персонал 9
місце розташування місце 9
розташування місце розташування 9
пішки площі ринок 8
сніданок шведський стіл 8
зручне розташування зручне 8
розташування зручне розташування 8
готель знаходиться центрі 7
привітний персонал чистота 7
місце розташування привітний 7
речі камері схову 7
привітний персонал зручне 7
персонал місце розташування 7
